In [16]:
import numpy as np
import cv2
import dlib
import matplotlib.pyplot as plt
import matplotlib.image as img
import math
from copy import deepcopy

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [10]:
# 얼굴 감지와 특징점 추정
def face(img):
    
    landmarks = []
    option = 0
    
    #dlib 사용
    pro_face, scores, idx = detector.run(img)
    dl_cnt = len(pro_face)
    score = 0
    
    if dl_cnt == 1:
        for f in pro_face:
            land = predictor(img, f)
            face_img = img[f.top():f.bottom(), f.left():f.right()].copy()
            score = scores[0]

            for l in land.parts():
                landmarks.append([l.x, l.y])
        option = 1
    elif dl_cnt > 1:
        num = 0
        for i, sco in enumerate(scores):
            if sco > score:
                score = sco
                num = i
        for i, f in enumerate(pro_face):
            if i == num:
                land = predictor(img, f)
                face_img = img[f.top():f.bottom(), f.left():f.right()].copy()

                for l in land.parts():
                    landmarks.append([l.x, l.y])
        option = 1
    else:
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        filtered_img = cv2.GaussianBlur(gray, (121, 121), 15)

        [faces, neighbours, weights] = faceCascade.detectMultiScale3(filtered_img, scaleFactor=1.05, minNeighbors=6 ,flags=cv2.CASCADE_SCALE_IMAGE , minSize=(100,100), outputRejectLevels = True)
        ha_cnt = len(faces)
        
        if ha_cnt == 1:
            for (x, y, w, h) in faces:
                dlib_rect = dlib.rectangle(int(x), int(y), int(x+w), int(y+h))
                face_img = img[int(y):int(y+h), int(x):int(x+w)].copy()
                
                land = predictor(img, dlib_rect)
                for l in land.parts():
                    landmarks.append([l.x, l.y])

                # landmarks = np.matrix([[p.x, p.y] for p in predictor(gray, dlib_rect).parts()])
            score = weights[0]
            option = 2
        elif ha_cnt > 1:
            num = 0
            for i, sco in enumerate(weights):
                if sco > score:
                    score = sco
                    num = i
            for i, (x, y, w, h) in enumerate(pro_face):
                if i == num:
                    dlib_rect = dlib.rectangle(int(x), int(y), int(x+w), int(y+h))
                    face_img = img[int(y):int(y+h), int(x):int(x+w)].copy()

                    landmarks = np.matrix([[p.x, p.y] for p in predictor(gray, dlib_rect).parts()])
            option = 2       
        else:
            face_img = img.copy()
            score = -1
    
    return landmarks, face_img

In [12]:
def point_dist(a, b):
    dist = math.sqrt(pow(a[0] - b[0], 2)+pow(a[1] - b[1], 2))
    return dist

In [4]:
def turn_direction(landmarks):
    left_cheek = point_dist(landmarks[2], landmarks[30])
    right_cheek = point_dist(landmarks[30], landmarks[16])
    
    if left_cheek > right_cheek:
        return 1 # 오른쪽으로 돌아간 얼굴
    else:
        return 2 # 왼쪽으로 돌아간 얼굴

In [8]:
def mat(pro_landmarks):
    direction = turn_direction(pro_landmarks)
    del_list = [False for i in range(68)]
    cor_landmarks = deepcopy(pro_landmarks)
    center = cor_landmarks[27]

    if direction == 1:

        # 턱 라인
        for i in range(0, 8):
            cor_landmarks[16 - i][1] = cor_landmarks[i][1]
            cor_landmarks[16 - i][0] = cor_landmarks[i][0] + 2 * (center[0] - cor_landmarks[i][0])
            del_list[i] = True
        cor_landmarks[8][0] = center[0]

        # 눈썹
        for i in range(17, 22):
            cor_landmarks[43 - i][1] = cor_landmarks[i][1]
            cor_landmarks[43 - i][0] = cor_landmarks[i][0] + 2 * (center[0] - cor_landmarks[i][0])
            del_list[i] = True

        # 콧대
        for i in range(27, 31):
            cor_landmarks[i][0] = center[0]

        # 콧볼
        for i in range(31, 33):
            cor_landmarks[66 - i][1] = cor_landmarks[i][1]
            cor_landmarks[66 - i][0] = cor_landmarks[i][0] + 2 * (center[0] - cor_landmarks[i][0])
            del_list[i] = True
        cor_landmarks[33][0] = center[0]

        # 눈
        for i in range(36, 40):
            cor_landmarks[81 - i][1] = cor_landmarks[i][1]
            cor_landmarks[81 - i][0] = cor_landmarks[i][0] + 2 * (center[0] - cor_landmarks[i][0])
            del_list[i] = True
        for i in range(40, 42):
            cor_landmarks[87 - i][1] = cor_landmarks[i][1]
            cor_landmarks[87 - i][0] = cor_landmarks[i][0] + 2 * (center[0] - cor_landmarks[i][0])
            del_list[i] = True

        # 입
        for i in range(48, 51):
            cor_landmarks[102 - i][1] = cor_landmarks[i][1]
            cor_landmarks[102 - i][0] = cor_landmarks[i][0] + 2 * (center[0] - cor_landmarks[i][0])
            del_list[i] = True
        cor_landmarks[51][0] = center[0]
        for i in range(58, 60):
            cor_landmarks[114 - i][1] = cor_landmarks[i][1]
            cor_landmarks[114 - i][0] = cor_landmarks[i][0] + 2 * (center[0] - cor_landmarks[i][0])
            del_list[i] = True
        cor_landmarks[59][0] = center[0]
        for i in range(60, 62):
            cor_landmarks[124 - i][1] = cor_landmarks[i][1]
            cor_landmarks[124 - i][0] = cor_landmarks[i][0] + 2 * (center[0] - cor_landmarks[i][0])
            del_list[i] = True
        cor_landmarks[62][0] = center[0]
        cor_landmarks[67][1] = cor_landmarks[65][1]
        cor_landmarks[67][0] = cor_landmarks[65][0] + 2 * (center[0] - cor_landmarks[65][0])
        del_list[65] = True
        cor_landmarks[66][0] = center[0]

        idx = 0
        for i in del_list:
            if i:
                del cor_landmarks[idx]
                del pro_landmarks[idx]
            else:
                idx = idx + 1

    # 얼굴이 왼쪽으로 돌아간 경우    
    if direction == 2:

        # 턱 라인
        for i in range(0, 8):
            cor_landmarks[i][1] = cor_landmarks[16 - i][1]
            cor_landmarks[i][0] = cor_landmarks[16 - i][0] + 2 * (center[0] - cor_landmarks[16 - i][0])
            del_list[16 - i] = True
        cor_landmarks[8][0] = center[0]

        # 눈썹
        for i in range(17, 22):
            cor_landmarks[i][1] = cor_landmarks[43 - i][1]
            cor_landmarks[i][0] = cor_landmarks[43 - i][0] + 2 * (center[0] - cor_landmarks[43 - i][0])
            del_list[43 - i] = True

        # 콧대
        for i in range(27, 31):
            cor_landmarks[i][0] = center[0]

        # 콧볼
        for i in range(31, 33):
            cor_landmarks[i][1] = cor_landmarks[66 - i][1]
            cor_landmarks[i][0] = cor_landmarks[66 - i][0] + 2 * (center[0] - cor_landmarks[66 - i][0])
            del_list[66 - i] = True
        cor_landmarks[33][0] = center[0]

        # 눈
        for i in range(36, 40):
            cor_landmarks[i][1] = cor_landmarks[81 - i][1]
            cor_landmarks[i][0] = cor_landmarks[81 - i][0] + 2 * (center[0] - cor_landmarks[81 - i][0])
            del_list[81 - i] = True
        for i in range(40, 42):
            cor_landmarks[i][1] = cor_landmarks[87 - i][1]
            cor_landmarks[i][0] = cor_landmarks[87 - i][0] + 2 * (center[0] - cor_landmarks[87 - i][0])
            del_list[87 - i] = True

        # 입
        for i in range(48, 51):
            cor_landmarks[i][1] = cor_landmarks[102 - i][1]
            cor_landmarks[i][0] = cor_landmarks[102 - i][0] + 2 * (center[0] - cor_landmarks[102 - i][0])
            del_list[102 - i] = True
        cor_landmarks[51][0] = center[0]
        for i in range(58, 60):
            cor_landmarks[i][1] = cor_landmarks[114 - i][1]
            cor_landmarks[i][0] = cor_landmarks[114 - i][0] + 2 * (center[0] - cor_landmarks[114 - i][0])
            del_list[114 - i] = True
        cor_landmarks[59][0] = center[0]
        for i in range(60, 62):
            cor_landmarks[i][1] = cor_landmarks[124 - i][1]
            cor_landmarks[i][0] = cor_landmarks[124 - i][0] + 2 * (center[0] - cor_landmarks[124 - i][0])
            del_list[124 - i] = True
        cor_landmarks[62][0] = center[0]
        cor_landmarks[65][1] = cor_landmarks[67][1]
        cor_landmarks[65][0] = cor_landmarks[67][0] + 2 * (center[0] - cor_landmarks[67][0])
        del_list[67] = True
        cor_landmarks[66][0] = center[0]

        idx = 0
        for i in del_list:
            if i:
                del cor_landmarks[idx]
                del pro_landmarks[idx]
            else:
                idx = idx + 1
    pro_points = np.array(pro_landmarks)
    cor_points = np.array(cor_landmarks)

    return (pro_points, cor_points)

In [21]:
# 입력
profile_img = cv2.imread('profile_1.jpg')
before_img = profile_img.copy()

# 얼굴 감지, 특징점 추정
pro_landmarks, pro_img = face(profile_img)

# 
pro_points, cor_points = mat(pro_landmarks)

# Homography 변환 행렬구하기
H, _ = cv2.findHomography(pro_points, cor_points, cv2.RANSAC)

# 변환
result = cv2.warpPerspective(profile_img, H, (profile_img.shape[1], profile_img.shape[0]))

In [ ]:
fig = plt.figure(figsize = (20, 35))
rows = 1
cols = 2

ax1 = fig.add_subplot(rows, cols, 1)
ax1.imshow(cv2.cvtColor(before_img, cv2.COLOR_BGR2RGB))
ax1.set_title('Before')
ax1.axis("off")


ax2 = fig.add_subplot(rows, cols, 2)
ax2.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
ax2.set_title('After')
ax2.axis("off")